In [19]:
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import MapType

In [20]:
url=" https://random-data-api.com/api/v2/users?size=100"

response=requests.get(url=url)
res_json=response.json()

In [21]:
spark=SparkSession.builder.appName("mycompany dataframe").config("spark.driver.extraClassPath","mysql-connector-j-8.1.0.jar").config('spark.jars.packages',"com.mysql:mysql-connector-j:8.1.0").getOrCreate()

In [22]:
df=spark.createDataFrame(res_json)

In [23]:
column_for_normal=[]
column_for_maptype=[]

for x in range(len(df.columns)):
    if isinstance(df.schema[df.columns[x]].dataType,MapType):
        column_for_maptype.append(df.columns[x])
    else:
        column_for_normal.append(df.columns[x])


In [24]:
print(column_for_maptype)
print(column_for_normal)

['address', 'credit_card', 'employment', 'subscription']
['avatar', 'date_of_birth', 'email', 'first_name', 'gender', 'id', 'last_name', 'password', 'phone_number', 'social_insurance_number', 'uid', 'username']


In [25]:
final_df=df.select(*column_for_normal)

In [26]:
for a in range(len(column_for_maptype)):
    s=df.select("*",explode(column_for_maptype[a])).groupBy("id").pivot("key").agg(first("value"))
    final_df=final_df.join(s,"id")
    

In [28]:
final_df.write\
.format("jdbc")\
.option("driver","com.mysql.cj.jdbc.Driver")\
.option("url","jdbc:mysql://127.0.0.1:3306/gh")\
.option("dbtable","company")\
.option("user","root")\
.option("password","root")\
.save()